In [ ]:
import sys
import os
import re
sys.path.insert(0, 'src/')

import pandas as pd
from src.table_utils import (
collect_scores_into_dict_miss,
extract_same_different_dataframes,\
ood_detection_pairs_,
aggregate_over_measures,
)
from IPython.display import display
pd.set_option('display.max_rows', None)

In [2]:
full_mis_rocauc = pd.read_csv('./tables/full_mis_rocauc.csv', index_col=0)

In [3]:
full_mis_rocauc = full_mis_rocauc[~full_mis_rocauc.UQMetric.str.endswith('Inner Inner')]

full_mis_rocauc.loc[(full_mis_rocauc.RiskType == "Bayes") & full_mis_rocauc.UQMetric.str.endswith("Outer"), "RiskType"] = 'Bayes Outer'
full_mis_rocauc.loc[(full_mis_rocauc.RiskType == "Bayes") & full_mis_rocauc.UQMetric.str.endswith("Inner"), "RiskType"] = 'Bayes Inner'

full_mis_rocauc.loc[(full_mis_rocauc.RiskType == "Total") & full_mis_rocauc.UQMetric.str.endswith("Outer"), "RiskType"] = 'Total Outer'
full_mis_rocauc.loc[(full_mis_rocauc.RiskType == "Total") & full_mis_rocauc.UQMetric.str.endswith("Inner"), "RiskType"] = 'Total Inner'

In [4]:
# trunc_df = full_mis_rocauc[
# ~full_mis_rocauc.RiskType.isin(['Bias', 'MV', 'MVBI', 'BiasBI', 'Bregman Information', 'Reverse Bregman Information'])
# # full_mis_rocauc.base_rule.isin(['Brier', 'Logscore', 'Spherical']) &
# # full_mis_rocauc.LossFunction.isin(['Brier', 'Logscore', 'Spherical']) &
# # ~(np.isclose(full_mis_rocauc.RocAucScore, np.float64(0.5)))
# ]

# # trunc_df.sort_values(by='RocAucScore')

# trunc_df.to_csv(os.path.join('tables', 'full_mis_rocauc_only_risks.csv'), index=False)

In [5]:
grouped_df = extract_same_different_dataframes(
    dataframe_=full_mis_rocauc,
)

In [6]:
same_dict, _ = collect_scores_into_dict_miss(
    dataframes_list_=[
        grouped_df.logscore_logscore,
        grouped_df.brier_brier,
        grouped_df.spherical_spherical,
    ],
)
same_df = pd.DataFrame.from_dict(same_dict)

same_agg_df = aggregate_over_measures(
    dataframe_=same_df,
    agg_func_='mean',
    by_=['InD'],
)

In [7]:
different_dict, _ = collect_scores_into_dict_miss(
    dataframes_list_=[
        grouped_df.logscore_not_logscore,
        grouped_df.brier_not_brier,
        grouped_df.spherical_not_spherical,
    ],
)
different_df = pd.DataFrame.from_dict(different_dict)

different_agg_df = aggregate_over_measures(
    dataframe_=different_df,
    agg_func_='mean',
    by_=['InD'],
)

In [8]:
all_dict, _ = collect_scores_into_dict_miss(
    dataframes_list_=[
        full_mis_rocauc,
    ],
)
all_df = pd.DataFrame.from_dict(all_dict)

all_agg_df = aggregate_over_measures(
    dataframe_=all_df,
    agg_func_='mean',
    by_=['InD'],
)

In [9]:
display(all_agg_df)
display(same_agg_df)
display(different_agg_df)

,Total Outer,Total Inner,Bayes Inner,Bayes Outer,Excess,Bregman Information,Reverse Bregman Information,Expected Pairwise Bregman Information,Bias,MV,MVBI,BiasBI
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
InD,,,,,,,,,,,,
cifar10,0.943867,0.945516,0.945516,0.941451,0.931047,0.929610,0.929936,0.933595,0.662205,0.855112,0.937753,0.770500
cifar100,0.816082,0.861309,0.861309,0.853726,0.738732,0.747067,0.723813,0.745315,0.473130,0.694561,0.764976,0.576680
missed_class_cifar10,0.871961,0.928622,0.928622,0.943150,0.804109,0.822559,0.782694,0.807075,0.571222,0.735386,0.813818,0.657489
noisy_cifar10,0.773801,0.806473,0.806473,0.789345,0.710099,0.717666,0.701384,0.711248,0.521974,0.665631,0.730278,0.608523
noisy_cifar100,0.763325,0.817531,0.817531,0.816822,0.626465,0.637540,0.613546,0.628309,0.438118,0.586976,0.647479,0.502568


,Total Outer,Total Inner,Bayes Inner,Bayes Outer,Excess,Bregman Information,Reverse Bregman Information,Expected Pairwise Bregman Information,Bias,MV,MVBI,BiasBI
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
InD,,,,,,,,,,,,
cifar10,0.947925,0.947645,0.947645,0.943026,0.946466,0.946845,0.946003,0.946549,0.771472,0.946749,0.947046,0.944337
cifar100,0.868571,0.868113,0.868113,0.861227,0.829012,0.837389,0.819206,0.830441,0.641034,0.827994,0.835120,0.828769
missed_class_cifar10,0.892959,0.929867,0.929867,0.944739,0.832276,0.863370,0.808601,0.824859,0.670820,0.826791,0.842936,0.817788
noisy_cifar10,0.805282,0.814185,0.814185,0.805731,0.743538,0.747681,0.739790,0.743143,0.578071,0.747875,0.747956,0.735353
noisy_cifar100,0.822756,0.826230,0.826230,0.826634,0.722989,0.732837,0.712474,0.723656,0.562190,0.723853,0.729520,0.725489


,Total Outer,Total Inner,Bayes Inner,Bayes Outer,Excess,Bregman Information,Reverse Bregman Information,Expected Pairwise Bregman Information,Bias,MV,MVBI,BiasBI
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
InD,,,,,,,,,,,,
cifar10,0.942852,0.944984,0.944984,0.941057,0.927192,0.925301,0.925919,0.930357,0.634888,0.832203,0.935430,0.727040
cifar100,0.802960,0.859608,0.859608,0.851850,0.716161,0.724486,0.699965,0.724034,0.431154,0.661203,0.747440,0.513658
missed_class_cifar10,0.866712,0.928310,0.928310,0.942753,0.797068,0.812356,0.776218,0.802630,0.546323,0.712535,0.806538,0.617414
noisy_cifar10,0.765931,0.804545,0.804545,0.785249,0.701740,0.710162,0.691783,0.703274,0.507950,0.645070,0.725858,0.576816
noisy_cifar100,0.748468,0.815356,0.815356,0.814369,0.602334,0.613716,0.588814,0.604472,0.407100,0.552757,0.626969,0.446838


In [10]:
same_agg_df.index

Index(['cifar10', 'cifar100', 'missed_class_cifar10', 'noisy_cifar10',
       'noisy_cifar100'],
      dtype='object', name='InD')

In [14]:
def enhance_latex_table(input_latex):
    lines = input_latex.split('\n')
    enhanced_lines = []

    for i, line in enumerate(lines):        
        if '\\toprule' in line:
            # Add multicolumn headers
            enhanced_lines.append(line)
            enhanced_lines.append(r'\multicolumn{2}{c}{Dataset} & \multicolumn{5}{c}{Metrics} \\')
            enhanced_lines.append(r'\cmidrule(lr){1-2} \cmidrule(lr){3-7}')
            continue

        # Add row coloring
        if '\\midrule' in line:
            enhanced_lines.append(line)
            enhanced_lines.append(r'\rowcolor{gray!10}')
        elif '\\bottomrule' in line:
            enhanced_lines.append(r'\end{tabular}')
        else:
            enhanced_lines.append(line)

    
    return '\n'.join(enhanced_lines)

def get_nice_df(df_):
    df_.index = pd.Index(data=['CIFAR10', 'CIFAR100', 'Missed class CIFAR10', 'Noisy CIFAR10',
       'Noisy CIFAR100'], name="InD")
    df_.columns = [
                # 'Bayes',
                # 'Excess',
                # 'Total',
                'Bayes(O)',
                'Bayes(I)',
                'Total(O)',
                'Total(I)',
                'BI',
                'RBI',
                'EPBI',
                # 'Bias',
                # 'MV',
                # 'MVBI',
                # 'BiasBI',
    ]
    # df_ = df_[['Bayes', 'Excess', 'Total', 'BI', 'RBI']]
    df_ = (100 * df_).round(2)

    display(df_)

    return df_, df_.to_latex(float_format="%.2f")

In [15]:
measures = [
    'Bayes Outer',
    'Bayes Inner',
    'Total Outer',
    'Total Inner',
    'Bregman Information',
    'Reverse Bregman Information',
    'Expected Pairwise Bregman Information']

# measures = ['Bayes', 'Excess', 'Total', 'Bregman Information', 'Reverse Bregman Information', 'Expected Pairwise Bregman Information']

In [16]:
nice_same = get_nice_df(same_agg_df.copy()[
                        measures
                        ])
print(enhance_latex_table(nice_same[1]))

,Bayes(O),Bayes(I),Total(O),Total(I),BI,RBI,EPBI
InD,,,,,,,
CIFAR10,94.30,94.76,94.79,94.76,94.68,94.60,94.65
CIFAR100,86.12,86.81,86.86,86.81,83.74,81.92,83.04
Missed class CIFAR10,94.47,92.99,89.30,92.99,86.34,80.86,82.49
Noisy CIFAR10,80.57,81.42,80.53,81.42,74.77,73.98,74.31
Noisy CIFAR100,82.66,82.62,82.28,82.62,73.28,71.25,72.37


\begin{tabular}{lrrrrrrr}
\toprule
\multicolumn{2}{c}{Dataset} & \multicolumn{5}{c}{Metrics} \\
\cmidrule(lr){1-2} \cmidrule(lr){3-7}
 & Bayes(O) & Bayes(I) & Total(O) & Total(I) & BI & RBI & EPBI \\
InD &  &  &  &  &  &  &  \\
\midrule
\rowcolor{gray!10}
CIFAR10 & 94.30 & 94.76 & 94.79 & 94.76 & 94.68 & 94.60 & 94.65 \\
CIFAR100 & 86.12 & 86.81 & 86.86 & 86.81 & 83.74 & 81.92 & 83.04 \\
Missed class CIFAR10 & 94.47 & 92.99 & 89.30 & 92.99 & 86.34 & 80.86 & 82.49 \\
Noisy CIFAR10 & 80.57 & 81.42 & 80.53 & 81.42 & 74.77 & 73.98 & 74.31 \\
Noisy CIFAR100 & 82.66 & 82.62 & 82.28 & 82.62 & 73.28 & 71.25 & 72.37 \\
\end{tabular}
\end{tabular}



In [17]:
nice_different = get_nice_df(different_agg_df.copy()[measures])
print(enhance_latex_table(nice_different[1]))

,Bayes(O),Bayes(I),Total(O),Total(I),BI,RBI,EPBI
InD,,,,,,,
CIFAR10,94.11,94.50,94.29,94.50,92.53,92.59,93.04
CIFAR100,85.19,85.96,80.30,85.96,72.45,70.00,72.40
Missed class CIFAR10,94.28,92.83,86.67,92.83,81.24,77.62,80.26
Noisy CIFAR10,78.52,80.45,76.59,80.45,71.02,69.18,70.33
Noisy CIFAR100,81.44,81.54,74.85,81.54,61.37,58.88,60.45


\begin{tabular}{lrrrrrrr}
\toprule
\multicolumn{2}{c}{Dataset} & \multicolumn{5}{c}{Metrics} \\
\cmidrule(lr){1-2} \cmidrule(lr){3-7}
 & Bayes(O) & Bayes(I) & Total(O) & Total(I) & BI & RBI & EPBI \\
InD &  &  &  &  &  &  &  \\
\midrule
\rowcolor{gray!10}
CIFAR10 & 94.11 & 94.50 & 94.29 & 94.50 & 92.53 & 92.59 & 93.04 \\
CIFAR100 & 85.19 & 85.96 & 80.30 & 85.96 & 72.45 & 70.00 & 72.40 \\
Missed class CIFAR10 & 94.28 & 92.83 & 86.67 & 92.83 & 81.24 & 77.62 & 80.26 \\
Noisy CIFAR10 & 78.52 & 80.45 & 76.59 & 80.45 & 71.02 & 69.18 & 70.33 \\
Noisy CIFAR100 & 81.44 & 81.54 & 74.85 & 81.54 & 61.37 & 58.88 & 60.45 \\
\end{tabular}
\end{tabular}



In [15]:
print(enhance_latex_table(pd.concat([nice_same[0], nice_different[0]], axis=1).to_latex(float_format="%.2f")))

\begin{tabular}{lrrrrrr}
\toprule
\multicolumn{2}{c}{Dataset} & \multicolumn{5}{c}{Metrics} \\
\cmidrule(lr){1-2} \cmidrule(lr){3-7}
 & Bayes & Excess & Total & Bayes & Excess & Total \\
InD &  &  &  &  &  &  \\
\midrule
\rowcolor{gray!10}
CIFAR10 & 94.53 & 94.65 & 94.78 & 94.30 & 92.72 & 94.39 \\
CIFAR100 & 86.47 & 82.90 & 86.83 & 85.57 & 71.62 & 83.13 \\
Missed class CIFAR10 & 93.73 & 83.23 & 91.14 & 93.55 & 79.71 & 89.75 \\
Noisy CIFAR10 & 81.00 & 74.35 & 80.97 & 79.49 & 70.17 & 78.52 \\
Noisy CIFAR100 & 82.64 & 72.30 & 82.45 & 81.49 & 60.23 & 78.19 \\
\end{tabular}
\end{tabular}



In [18]:
nice_same[0].mean()

Bayes(O)    87.624
Bayes(I)    87.720
Total(O)    86.752
Total(I)    87.720
BI          82.562
RBI         80.522
EPBI        81.372
dtype: float64

In [19]:
nice_same[0].std()

Bayes(O)    6.482648
Bayes(I)    5.997137
Total(O)    5.697049
Total(I)    5.997137
BI          8.794568
RBI         9.069544
EPBI        8.821072
dtype: float64

In [20]:
nice_different[0].mean()

Bayes(O)    86.708
Bayes(I)    87.056
Total(O)    82.540
Total(I)    87.056
BI          75.722
RBI         73.654
EPBI        75.296
dtype: float64

In [21]:
nice_different[0].std()

Bayes(O)     7.232294
Bayes(I)     6.403494
Total(O)     7.979279
Total(I)     6.403494
BI          11.744700
RBI         12.509404
EPBI        12.175206
dtype: float64

In [12]:
same_agg_df.eq(same_agg_df.max(axis=1), axis=0)

,Bayes,Excess,Total,Bregman Information,Reverse Bregman Information,Bias,MV,MVBI,BiasBI
,mean,mean,mean,mean,mean,mean,mean,mean,mean
InD,,,,,,,,,
cifar10,False,False,True,False,False,False,False,False,False
cifar100,False,False,True,False,False,False,False,False,False
missed_class_cifar10,True,False,False,False,False,False,False,False,False
noisy_cifar10,True,False,False,False,False,False,False,False,False
noisy_cifar100,True,False,False,False,False,False,False,False,False


In [13]:
different_agg_df.eq(different_agg_df.max(axis=1), axis=0)

,Bayes,Excess,Total,Bregman Information,Reverse Bregman Information,Bias,MV,MVBI,BiasBI
,mean,mean,mean,mean,mean,mean,mean,mean,mean
InD,,,,,,,,,
cifar10,False,False,True,False,False,False,False,False,False
cifar100,True,False,False,False,False,False,False,False,False
missed_class_cifar10,True,False,False,False,False,False,False,False,False
noisy_cifar10,True,False,False,False,False,False,False,False,False
noisy_cifar100,True,False,False,False,False,False,False,False,False


In [14]:
all_agg_df.eq(all_agg_df.max(axis=1), axis=0)

,Bayes,Excess,Total,Bregman Information,Reverse Bregman Information,Bias,MV,MVBI,BiasBI
,mean,mean,mean,mean,mean,mean,mean,mean,mean
InD,,,,,,,,,
cifar10,False,False,True,False,False,False,False,False,False
cifar100,True,False,False,False,False,False,False,False,False
missed_class_cifar10,True,False,False,False,False,False,False,False,False
noisy_cifar10,True,False,False,False,False,False,False,False,False
noisy_cifar100,True,False,False,False,False,False,False,False,False


In [15]:
(same_agg_df - different_agg_df) > 0

,Bayes,Excess,Total,Bregman Information,Reverse Bregman Information,Bias,MV,MVBI,BiasBI
,mean,mean,mean,mean,mean,mean,mean,mean,mean
InD,,,,,,,,,
cifar10,True,True,True,True,True,True,True,True,True
cifar100,True,True,True,True,True,True,True,True,True
missed_class_cifar10,True,True,True,True,True,True,True,True,True
noisy_cifar10,True,True,True,True,True,True,True,True,True
noisy_cifar100,True,True,True,True,True,True,True,True,True
